In [1]:
!pip install "tensorflow>=1.7.0"
!pip install tensorflow-hub

In [2]:
import tensorflow_hub as hub
import tensorflow as tf  # uses the default tensorflow currently 1.x 

print(tf.test.gpu_device_name())
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf.device)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

/device:GPU:0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7129215868516016883
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14995855787612750204
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13771941622950872137
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11330115994
locality {
  bus_id: 1
  links {
  }
}
incarnation: 303691912972855955
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]
<function device at 0x7f35c7a261e0>
Num GPUs Available:  1


In [0]:
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [0]:
path = untar_data(URLs.IMDB_SAMPLE)

In [6]:
df = pd.read_csv(path/'texts.csv')
print(df.head())
print(df.shape)

      label                                               text  is_valid
0  negative  Un-bleeping-believable! Meg Ryan doesn't even ...     False
1  positive  This is a extremely well-made film. The acting...     False
2  negative  Every once in a long while a movie will come a...     False
3  positive  Name just says it all. I watched this movie wi...     False
4  negative  This movie succeeds at being one of the most u...     False
(1000, 3)


In [7]:
# split into train and test
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size = 0.2)
print(train_df.shape[0])
print(test_df.shape[0])

text_train   = np.array(train_df['text'])
labels_train = np.array(train_df['label'])

text_test   = np.array(test_df['text'])
labels_test = np.array(test_df['label'])

800
200


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df = 5)
count_features = vect.fit_transform(train_df['text'])  # sparse matrix
print(count_features.shape)

(800, 3279)


In [8]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
from keras.utils import to_categorical
from keras.models import Sequential
import keras

Using TensorFlow backend.


In [9]:
def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"] # default computes the mean of all three elmo layers

def build_model(): 
    input_text = Input(shape=(1,), dtype="string")
    embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
    dense = Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(embedding)
    pred = Dense(2, activation='softmax')(dense)
    model = Model(inputs=[input_text], outputs=pred)
    #model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model_elmo = build_model()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [10]:
print(model_elmo.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 514       
Total params: 262,914
Trainable params: 262,914
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
ind_positive = labels_train == 'positive'
ind_negative = labels_train == 'negative' 
labels_train[ind_positive] = 1 
labels_train[ind_negative] = 0

cat_train = to_categorical(labels_train)
print(labels_train[:10])

[1 1 0 0 0 1 1 1 0 0]


In [0]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [15]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model_elmo.fit(text_train, cat_train, epochs = 5, batch_size= 8 , validation_split = 0.2)
    model_elmo.save_weights('./model_elmo_weights.h5')

Train on 640 samples, validate on 160 samples
Epoch 1/5
640/640 [==============================] - 435s 680ms/step - loss: 0.9393 - acc: 0.6875 - val_loss: 0.8177 - val_acc: 0.6937
Epoch 2/5
640/640 [==============================] - 428s 668ms/step - loss: 0.7401 - acc: 0.7641 - val_loss: 0.7168 - val_acc: 0.7312
Epoch 3/5
640/640 [==============================] - 418s 653ms/step - loss: 0.6838 - acc: 0.7766 - val_loss: 0.6843 - val_acc: 0.7562
Epoch 4/5
640/640 [==============================] - 417s 652ms/step - loss: 0.7834 - acc: 0.7266 - val_loss: 0.6614 - val_acc: 0.7188
Epoch 5/5
640/640 [==============================] - 420s 656ms/step - loss: 0.5756 - acc: 0.8187 - val_loss: 0.6068 - val_acc: 0.7875


In [77]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    model_elmo = build_model() 
    model_elmo.load_weights('./model_elmo_weights.h5')
    import time
    t = time.time()
    predicts = model_elmo.predict(np.array(["this is a good movie", "this is a bad movie", "this is not a bad movie", "this is not a good movie", 
                                            "this is an ok movie", "I really liked this movie. bad movie", 
                                            "bad movie. But I really liked the ending"]))
    print("time: ", time.time() - t)
    print(predicts)
    output_class = np.zeros(predicts.shape[0], dtype = int)
    for i,p in enumerate(predicts):
      if p[0] <= p[1]:
        output_class[i] = 1
    print(output_class)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


time:  2.596433639526367
[[0.061242 0.938758]
 [0.709842 0.290158]
 [0.811621 0.188379]
 [0.554517 0.445483]
 [0.151817 0.848183]
 [0.397124 0.602876]
 [0.229242 0.770758]]
[1 0 0 0 1 1 1]


In [30]:
ind_positive = labels_test == 'positive'
ind_negative = labels_test == 'negative' 
labels_test[ind_positive] = 1 
labels_test[ind_negative] = 0

cat_test = to_categorical(labels_test)
print(labels_test[:10])
print(cat_test[:10])

[1 1 0 1 0 1 0 1 1 1]
[[0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [68]:
from sklearn.metrics import accuracy_score

with tf.Session() as session:
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    model_elmo = build_model() 
    model_elmo.load_weights('./model_elmo_weights.h5')
    predicts = np.zeros((len(text_test),2))
    for ind in range(0,len(text_test),2):
      predicts[ind:ind+2,:] = model_elmo.predict(np.array([text_test[ind], text_test[ind+1]]))
    # assign class
    output_class = np.zeros(predicts.shape[0], dtype = int)
    for i,p in enumerate(predicts):
      if p[0] <= p[1]:
        output_class[i] = 1

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


ValueError: ignored

In [76]:
accuracy = accuracy_score(np.array(labels_test, dtype = int), output_class)
print(accuracy)

import pandas as pd
print(pd.crosstab(np.array(labels_test, dtype = int), output_class))

0.79
col_0   0   1
row_0        
0      91  13
1      29  67
